In [1]:
import pandas as pd
import numpy as np
import dtale as dt



In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


In [3]:
torch.cuda.empty_cache()

In [4]:
torch.cuda.is_available()

True

In [5]:
y_train=pd.read_pickle('y_train.pkl')
y_test=pd.read_pickle('y_test.pkl')
x_test=pd.read_pickle('x_test.pkl')
x_train=pd.read_pickle('x_train.pkl')

In [6]:
pred_col='log_bearish_week'

In [7]:
test_y=y_test[pred_col]

In [8]:
training_data1=x_train

In [9]:
Y_train=y_train[pred_col]
#Y_train=df['Close']

# Data Loading


In [10]:
from torch.utils.data import Dataset, DataLoader
import joblib
class MyDataset(Dataset):
    def __init__(self, X, Y):
        #normalize

    
        self.X = torch.tensor(X, dtype=torch.float32)
        new_shape = (len(Y), 1)
        self.Y = torch.tensor(Y, dtype=torch.float32)
        self.Y = self.Y.view(new_shape)
        
    
        
        #self.Y = self.Y.view(new_shape)
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

# Normalize the X data
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

In [11]:
#scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = StandardScaler()
scaled_fit=scaler.fit_transform(training_data1)
        

joblib.dump(scaler, 'sc.joblib') 

['sc.joblib']

In [12]:
#scaler_y = StandardScaler()
scaled_fit_y=Y_train.values.reshape(-1, 1)
       

In [13]:
# Assuming X is a DataFrame with 5 columns and Y is a DataFrame with 1 column
#train_data = MyDataset(scaled_fit, Y_train.values.reshape(-1, 1))
train_data = MyDataset(scaled_fit, scaled_fit_y)

train_loader = DataLoader(train_data, shuffle=True, batch_size=32) #

In [14]:
training_data1

,log_normalized_change,log_price_range,log_price_range_high,log_price_range_low,log_premarket_changes,log_Smart_Money,log_volume_deviation,log_norm_avg_deviation_200,log_norm_avg_deviation_300,log_ha_change,log_ha_change_intra,log_ha_change_intra_high,log_ha_high_low
299,-0.018210,0.022816,0.006183,0.016633,-0.001845,-0.016364,0.594377,0.058461,0.094445,-0.011852,-0.015852,0.025832,0.025832
300,0.012927,0.013987,0.010706,0.003281,0.003214,0.009713,0.273865,0.070581,0.106799,-0.002194,-0.010152,0.017736,0.017736
301,0.001720,0.008260,0.006735,0.001525,-0.000728,0.002449,-0.231275,0.071486,0.107933,0.006602,0.001513,0.001932,0.008260
302,-0.010966,0.013219,0.005758,0.007461,-0.004173,-0.006793,0.517538,0.059751,0.096423,-0.005753,-0.004997,0.010349,0.013219
303,0.014857,0.015574,0.010378,0.005196,0.005664,0.009193,0.296428,0.073756,0.110679,0.004595,0.002093,0.006718,0.015574
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4262,0.000693,0.008479,0.005908,0.002571,0.002125,-0.001433,-0.172502,0.033980,-0.002405,0.008425,0.015805,0.000000,0.021232
4263,-0.015447,0.017550,0.000618,0.016932,-0.000124,-0.015324,0.208525,0.018386,-0.017319,-0.009913,-0.002042,0.011098,0.017550
4264,0.001631,0.007836,0.003308,0.004528,0.000301,0.001329,-0.173034,0.019831,-0.015107,-0.007116,-0.008137,0.012697,0.012697
4265,-0.018622,0.027653,0.004343,0.023310,0.002053,-0.020676,0.202728,0.001120,-0.033089,-0.006484,-0.010561,0.024035,0.027653


# Pytorch setup

In [15]:
# Define your LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout):
        super(LSTM,self).__init__()
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # Reshape the input tensor to match the expected shape of (batch_size, sequence_length, input_size)
        x = x.view(-1, 1, x.shape[-1])
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        output, (h_n, c_n) = self.lstm(x)
        # Apply activation function
        output = self.relu(output)
        output = self.dropout(output[:, -1, :])
        
        output = self.fc(output)
        output = self.sigmoid(output)
 
        return torch.squeeze(output)

In [16]:
num_epochs = 35000
learning_rate = 0.0005
num_layers=5
input_size = len(x_train.columns)
hidden_size = 90
output_size = 1
dropout=.5

In [17]:
def array_list_to_dataframe(arrays,length):
    # Convert the list of arrays to a single 2D array
    array_2d = np.concatenate(arrays, axis=1)
    
    # Reshape the 2D array into a 5-column array
    array_5col = np.reshape(array_2d, (-1, length))
    
    # Create a DataFrame from the 5-column array
    df = pd.DataFrame(array_5col)
    
    return df

In [18]:
certainty_2sd_counts=pd.DataFrame()
certainty_4sd_counts=pd.DataFrame()
certainty_5sd_counts=pd.DataFrame()
certainty_6sd_counts=pd.DataFrame()


In [19]:
#set up gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
for j in range(7):
    # Instantiate the model
    model = LSTM(input_size=input_size, hidden_size=hidden_size,num_layers=num_layers,output_size=1,dropout=dropout)#output_size=1
    model=torch.compile(model)
    model.to(device)
    train_data_x=train_data.X.to(device)
    train_data_y=train_data.Y.to(device)
    # Define the loss function and optimizer
    criterion = nn.BCELoss() 
    optimizer = torch.optim.Adam(model.parameters(), lr=0.003,weight_decay=6e-7) 
        # Train the model
    for epoch in range(num_epochs):
        outputs = model(train_data_x)
        loss = criterion(outputs, train_data_y.squeeze())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1000 == 0:
            print("Epoch: %d, loss: %1.5f"  % (epoch, loss.item()),j)
        """for batch_X, batch_Y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_Y.squeeze())
            loss.backward()
            optimizer.step()
            if epoch % 100 == 0:
                print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))"""
        
        
    #Transform test data
    transformedData = scaler.transform(x_test)
    transformedData_y=test_y.values.reshape(-1, 1)

    # Use the trained model for predictions
    
    
    
    test_data = MyDataset(transformedData, transformedData_y)
    test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
    test_data_x=test_data.X.to(device)
    
    outputlist=[]
    
    #testing data
    
    model.train()
    len_outputs=100
    with torch.no_grad():
        for i in range(len_outputs):
            output = model(test_data_x)
            outputlist.append(output)
    model.eval()
    
    scaled_output_list=[]
    for outputs in outputlist:
        scaled_output=outputs.cpu().reshape(-1, 1)
        scaled_output_list.append(scaled_output)
        
    model_test_outputs=array_list_to_dataframe(scaled_output_list,len_outputs )
    
    count1 = (model_test_outputs > .999).sum(axis=1)
    count2=(model_test_outputs > .9999).sum(axis=1)
    count3=(model_test_outputs > .99999).sum(axis=1)
    count4=(model_test_outputs > .999999).sum(axis=1)
    
    
    column_name = 'Column ' + str(j)
    
    certainty_2sd_counts[column_name] =count1 
    certainty_4sd_counts[column_name] =count2
    certainty_5sd_counts[column_name] =count3
    certainty_6sd_counts[column_name] =count4

/home/jeffwa/anaconda3/envs/DL_new/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



Epoch: 0, loss: 0.67769 0
Epoch: 1000, loss: 0.21200 0
Epoch: 2000, loss: 0.11343 0
Epoch: 3000, loss: 0.03641 0
Epoch: 4000, loss: 0.01845 0
Epoch: 5000, loss: 0.00243 0
Epoch: 6000, loss: 0.00035 0
Epoch: 7000, loss: 0.00044 0
Epoch: 8000, loss: 0.03899 0
Epoch: 9000, loss: 0.00169 0
Epoch: 10000, loss: 0.00188 0
Epoch: 11000, loss: 0.00121 0
Epoch: 12000, loss: 0.00027 0
Epoch: 13000, loss: 0.00026 0
Epoch: 14000, loss: 0.00011 0
Epoch: 15000, loss: 0.00013 0
Epoch: 16000, loss: 0.00014 0
Epoch: 17000, loss: 0.00002 0
Epoch: 18000, loss: 0.00048 0
Epoch: 19000, loss: 0.00003 0
Epoch: 20000, loss: 0.00026 0
Epoch: 21000, loss: 0.00003 0
Epoch: 22000, loss: 0.00007 0
Epoch: 23000, loss: 0.00004 0
Epoch: 24000, loss: 0.00003 0
Epoch: 25000, loss: 0.00028 0
Epoch: 26000, loss: 0.00001 0
Epoch: 27000, loss: 0.00026 0
Epoch: 28000, loss: 0.00002 0
Epoch: 29000, loss: 0.00001 0
Epoch: 30000, loss: 0.00002 0
Epoch: 31000, loss: 0.00004 0
Epoch: 32000, loss: 0.00002 0
Epoch: 33000, loss: 0.0

In [25]:
certainty_6sd_counts.mean(axis=1)

0      0.000000
1      2.857143
2     48.571429
3     26.428571
4      0.000000
5      0.428571
6      0.000000
7      0.000000
8      0.000000
9     15.428571
10     0.000000
11    40.000000
dtype: float64

In [22]:
certainty_6sd_counts

,Column 0,Column 1,Column 2,Column 3,Column 4,Column 5,Column 6
0,0,0,0,0,0,0,0
1,0,0,20,0,0,0,0
2,77,0,90,30,0,50,93
3,0,0,98,26,61,0,0
4,0,0,0,0,0,0,0
5,0,0,0,3,0,0,0
6,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0
9,0,0,5,76,0,27,0
